## Imports and seeds

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np
import math
import random
from matplotlib import pyplot as plt
import torch
import gpytorch
from tqdm import tqdm
import pandas as pd
from sklearn.linear_model import Ridge
from scipy.interpolate import griddata
from multitask_basis_func_gp.gen_funcs import *
from multitask_basis_func_gp.basis_funcs import *
from multitask_basis_func_gp.utils import *
from multitask_basis_func_gp.visual_utils import *
from multitask_basis_func_gp.multitask_basis_func_gp import MultitaskBasisFuncGPModel
from multitask_basis_func_gp.dot_product_prediction_strategy import *

from gpytorch.distributions import MultitaskMultivariateNormal, MultivariateNormal
from gpytorch.means import ConstantMean
from gpytorch.kernels import MaternKernel

In [ ]:
import os
notebook_path = os.path.abspath("")

In [ ]:
# Set seeds

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

## Set up

Recall that we model the KPI's dependence on time $t$ (could be day/hour/week) and dual $d$ as KPI(t, d) = SUM coeff(t) * basis_func(d) + noise.

That is, $KPI(t,d) = \sum_{i=1}^F \alpha_i(t)f_i(d) + \epsilon$ for deterministic monotonic basis functions $f_i(d)$ with $i = 1 \to F$ and iid scalar Gaussian noise $\epsilon \sim N(0, \sigma_{up}^2)$. We model the F-dimensional vector of coefficients $(\alpha_i(t))_i$ as a GP, denote by coeffs or coeff in the code. We will first fit a prior using Excalibur curves and then use actual historical data to make updates. 

IF we choose dual $d_0$ at time $t_0$, then we observe only $KPI(t_0, d_0)$, we do not immediately have access to good counterfactuals. So we want to update the WHOLE F-dimensional vector-valued GP $t \mapsto [\alpha_i(t)]_i$ using information about ONLY its dot product with the vector $[f_i(d_0)]_i$ at time $t_0$, because this is exactly $KPI(t_0, d_0) + \epsilon$.

### Declare major parameters

In [ ]:
# Various hand-picked parameters

# Important parameters
max_time = 24 # CHANGE TO 7 OR 0 IF WORKING WITH DAYS
train_t = torch.arange(max_time).float() # Setting all times to be used for training data
use_relu = True # Whether to use ReLU to make the occasional negative coefficients positive, or to rely on the actual prediction itself

# update_sigma is the variance for the GP update, same as sigma_up
y_scale = 1.0 # Scale of the output data, typically 1.0
init_update_sigma = 0.1*y_scale # Initial value for the noise in the GP update, typically 0.1*y_scale
# init_update_sigma = 0.8

"""Notes on update_sigma:
update_sigma is currently set by hand in the notebook, change to automated grid search later
Recall this is the stddev of epsilon, the scalar noise
KEEP IN MIND: update_sigma controls how much faith in signal vs faith in prior model we have
1/500*kpi_max to kpi_max, typical range for update_sigma
I suspect that best results will be between 0.01*kpi_max and 0.3*kpi_max
"""

# For optimization
verbose = True # SET TO FALSE WHEN DEPLOYING
per_compute_verbose = False
# Parameters for prior hyperparameter gradient descent 
prior_lr = 0.1
prior_sched_gamma = 0.999
# Parameters for posterior hyperparameter gradient descent 
post_lr = 0.1
post_sched_gamma = 0.999

smoke_test = ('CI' in os.environ)
num_prior_train_iter = 2 if smoke_test else 50
num_post_train_iter = 50

# More technical parameters
dual_scale_factor = 6 # Scale duals by dual_scale_factor*dual_max to allow wiggle room for extrapolation beyond dual_max
grid_size = 500 # Grid of duals used to generated interpolated data
num_basis = 45 # number of basis functions, this is F


rank = 45 # Rank of covariance matrix for coefficients alpha 
""" Notes on rank: In practice, it doesn't seem to matter much.
I think this is the rank you get after hyperparameter optimization in gpytorch and before using excalibur data.
It's not the rank you would get after prior fitting (which is in our case GP inference over direct coefficients learnt from excalibur)
"""

# Choose mean and covariance module for the vector-valued coeff GP
mean_module = gpytorch.means.MultitaskMean(ConstantMean(), num_tasks=num_basis)
covar_module = gpytorch.kernels.MultitaskKernel(MaternKernel(), num_tasks=num_basis, rank=rank)
# If we want cyclic relation across hours, then we might want a cyclic distance function for the kernel

# Parameters for discrete sigmoid basis functions
# ONLY change if you are changing the basis functions and have looked into how to do that
gap = 0.02 # Size of non flat part of discrete sigmoid functions, slightly less than 1/num_basis

In [ ]:
# Set up basis functions
start_num = num_basis
d_start_range = np.linspace(0, 1, start_num)
steep_d_sigmoid_basis = BasisFuncs(discrete_sigmoid_gap_based)
for start in d_start_range:
    steep_d_sigmoid_basis.append([start, gap])

### Set up training coefficients and initialize model

In [ ]:
# Get training coefficients
# We use linear regression over dual-kpi grid to get training coefficients coeff_i(t) for each time t
"""IMPORTANT UTILITY FUNCTION"""
train_t, train_coeffs = gen_true_coeffs(df_exc, scale_df, inst_id, train_t, dual_scale_factor, grid_size, basis_funcs = steep_d_sigmoid_basis, kpi=kpi)

# Shape = num_train_times, num_basis
print(inst_id)

## Initialize model and train prior

In [ ]:
# Set up "likelihood" (essentially takes GP output distribution f(x) and gives an output distribution by adding appropriate noise)
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=num_basis)

# Initialize model
model = MultitaskBasisFuncGPModel(train_t, train_coeffs, num_basis, likelihood, init_update_sigma, steep_d_sigmoid_basis, max_time, mean_module, covar_module)

"""
You can add kpi_observed and dual_chosen and time_observed right here as well if you don't have train_coeffs. 
In that case, just skip hyperparameter optimization.
You can change mean_module to a custom mean and covar_module as well if you want, in case you have some general prior knowledge.
model.update_inputs = ...
model.update_basis_inputs = ...
model.update_label = ...
"""

In [ ]:
# this is for running the notebook in our testing framework
prior_hyperparam_optim(model, likelihood, train_t, train_coeffs, num_prior_train_iter, lr = prior_lr, scheduler_gamma = prior_sched_gamma, verbose=verbose)

In [ ]:
for name, param in model.named_parameters():
    param.requires_grad = False
model.log_update_sigma.requires_grad = True

### Check prior graphs against excalibur graphs (visualization only)

There will be difference in the power (magnitude) but not shape of the dual-kpi graphs from the prior and the excalibur curve data. That's because there is a high variation in power of the hourly dual-kpi curves as the hours progress, while the GP prior fit "smooths out" this variation. Yannis believes (and I agree) that this is because of poor calibration of excalibur curves, which only happens at the day level.

#### Dual-KPI graphs

In [ ]:
# Choose time to display dual-kpi curve for
prior_check_time = 5

# Get test data
model.eval()
likelihood.eval()

# Get kpi values for the scaled dual grid
scaled_dual_x, kpi_true, scale = gen_unif_grid_dual_kpi(df_exc, scale_df, inst_id, prior_check_time, dual_scale_factor, grid_size, kpi=kpi)
kpi_prior = gen_kpi_vals_single_time(model, likelihood, steep_d_sigmoid_basis, scaled_dual_x,\
                                     prior_check_time, kpi=kpi, mode = "prior", use_relu=use_relu).detach()

# Plot dual-kpi graph for chosen time
fig, ax = plt.subplots()
ax.plot(scaled_dual_x*scale, kpi_true, color = "orange")
# ax.plot(scaled_dual_x*scale, kpi_prior)
ax.set_xlabel("Dual (Scaled)")
ax.set_ylabel(kpi)
# ax.legend(["Excalibur Curve", "Prior Prediction"])
plt.title("Dual-KPI Excalibur Curve, Instance Id " + str(inst_id) + ", Time " + str(prior_check_time) + "")
p

#### Coeff-time graphs (sanity check for "power smoothing" due to GP)

In [ ]:
# Choose coefficient index to plot coeff-time graph for
check_coeffs_idx = 1

# Set eval mode for safety
model.eval()
likelihood.eval()

prior_coeffs = model(train_t).mean.detach().numpy()
likelihood(model(train_t))
show_coeff_time_graph(inst_id, train_t, prior_coeffs, train_coeffs, check_coeffs_idx,\
                      "Prior Coefficient Variation", "Excalibur Coefficient Variation", use_relu=use_relu)

## Set up update data and make posterior update

### Set up update data

In [ ]:
update_inputs, update_basis_inputs, update_labels =  get_real_vals(df_real, scale_df, inst_id, dual_scale_factor)
model.update_inputs, model.update_basis_inputs, model.update_labels = update_inputs, update_basis_inputs, update_labels

# Toggle to slice for smaller inputs

In [ ]:
# Generate posterior vals
check_time = 11

# Get kpi values corresponding to given dual grid
"""IMPORTANT UTILITY FUNCTION"""
kpi_post = gen_kpi_vals_single_time(model, likelihood, steep_d_sigmoid_basis, scaled_dual_x, check_time, kpi=kpi, mode = "posterior", use_relu=use_relu)
kpi_prior = gen_kpi_vals_single_time(model, likelihood, steep_d_sigmoid_basis, scaled_dual_x, check_time, kpi=kpi, mode = "prior", use_relu=use_relu)

# Get prediction for single dual used
dual_chosen = 0.4
dual_used = torch.tensor([dual_chosen], dtype=torch.float)
kpi_post_pred = gen_kpi_vals_single_time(model, likelihood, steep_d_sigmoid_basis, dual_used, check_time, kpi=kpi, mode = "posterior", use_relu=use_relu)

### Compare prior and posterior curves (visualization only)

In [ ]:
# Change update sigma if you want
model.log_update_sigma = torch.nn.Parameter(torch.tensor(math.log(0.5)))
model.update_inputs, model.update_basis_inputs, model.update_labels = update_inputs[:5], update_basis_inputs[:5], update_labels[:5]
# Shows prior and posterior dual-kpi curves
show_prior_post_dual_kpi_curves(model, likelihood, scale_df, inst_id, model.update_inputs, dual_scale_factor, grid_size, steep_d_sigmoid_basis, kpi=kpi, use_relu=use_relu)

### Find optimal update_sigma

In [ ]:
plt.close('all')
import gc
gc.collect()

# torch clear memory
torch.cuda.empty_cache()

In [ ]:
#try loop that tries to use gen_kpi_vals for various values of kpi_scale to determine init_update_sigma until no errors are thrown
from multitask_basis_func_gp.utils import *
min_kpi_scaling_power = -2
max_kpi_scaling_power = 1
kpi_scalings = torch.logspace(-3, 0, 15)

sigma_grid_size = 30
kpi_scaling = get_kpi_scaling(df_real, scale_df, inst_id, kpi_scale, model, likelihood, dual_scale_factor, steep_d_sigmoid_basis, kpi="Spend", use_relu = use_relu,\
                    min_power = min_kpi_scaling_power, kpi_scalings = kpi_scalings, verbose = True)

print(kpi_scaling)

#### Generate and plot error array for given update_sigma grid

In [ ]:
# Set sigma grid
sigma_grid = torch.logspace(min_kpi_scaling_power, max_kpi_scaling_power, sigma_grid_size)*kpi_scale*kpi_scaling

# Make sure update data is empty
empty_tensor = torch.tensor([], dtype=torch.float)
model.update_inputs, model.update_basis_inputs, model.update_labels = empty_tensor, empty_tensor, empty_tensor

"""IMPORTANT UTILITY FUNCTION"""
error_array = gen_error_array(None, df_real, scale_df, inst_id, model, likelihood,\
                sigma_grid, dual_scale_factor, steep_d_sigmoid_basis, \
                              kpi=kpi, use_relu = use_relu, verbose = verbose, per_compute_verbose =per_compute_verbose)

In [ ]:
# Set up update data
model.update_inputs, model.update_basis_inputs, model.update_labels =  get_real_vals(df_real, scale_df, inst_id, dual_scale_factor)

# Generate error array for historical training data
error_array_test = gen_error_array(df_real, df_real_test, scale_df, inst_id, model, likelihood,\
                sigma_grid, dual_scale_factor, steep_d_sigmoid_basis, \
                              kpi=kpi, use_relu = use_relu, verbose = verbose, per_compute_verbose =per_compute_verbose)

In [ ]:
# Plot sigma values against errors
fig, ax = plt.subplots()
ax.plot(torch.log10(sigma_grid), error_array.detach())
ax.plot(torch.log10(sigma_grid), error_array_test.detach())
ax.legend(["Training error", "Test error (next week)"])
ax.set_title("InstanceId " + str(inst_id))
ax.set_xlabel("update_sigma")
ax.set_ylabel("prediction_error")

In [ ]:
# Get sigma with min error
best_sigma = sigma_grid[torch.argmin(error_array)]
test_error = compute_post_pred_error(df_real, df_real_test, scale_df, inst_id, model, likelihood, best_sigma, dual_scale_factor, steep_d_sigmoid_basis, kpi=kpi, use_relu = use_relu, verbose = verbose)
print("Best sigma for training data: ", best_sigma)
print("Test error for best sigma: ", test_error)

#### Find optimal sigma using grid search

In [ ]:
# Set sigma grid
# PLAY AROUND WITH THIS CHOICE IN DIFFERENT SCENARIOS
sigma_grid = torch.logspace(-2, 1, 30)*kpi_scale*kpi_scaling

# Set up update data
model.update_inputs, model.update_basis_inputs, model.update_labels =  get_real_vals(df_real, scale_df, inst_id, dual_scale_factor)

# Directly get update_sigma with min error on real data
best_sigma = best_update_sigma_grid_search(None, df_real, scale_df, inst_id, model, likelihood,\
                sigma_grid, dual_scale_factor, steep_d_sigmoid_basis,\
               kpi=kpi, use_relu = use_relu, verbose = verbose, per_compute_verbose = per_compute_verbose)

In [ ]:
# Get sigma with min error
best_sigma = sigma_grid[torch.argmin(error_array)]
test_error = compute_post_pred_error(df_real, df_real_test, scale_df, inst_id, model, likelihood, best_sigma, dual_scale_factor, steep_d_sigmoid_basis, kpi=kpi, use_relu = use_relu, verbose = verbose)
print("Best sigma for training data: ", best_sigma)
print("Test error for best sigma: ", test_error)

#### Find optimal sigma using gradient based optimization

In [ ]:
# Set up empty update data
empty_tensor = torch.tensor([], dtype=torch.float)
model.update_inputs, model.update_basis_inputs, model.update_labels = empty_tensor, empty_tensor, empty_tensor

# Set initial update_sigma
init_update_sigma = kpi_scale*kpi_scaling

# Implicitly update model's update_sigma using gradient descent over the same MSE target used above
loss_array = best_update_sigma_grad_opt(None, df_real, scale_df, inst_id, model, likelihood, dual_scale_factor, steep_d_sigmoid_basis, 30,\
                          lr = post_lr, scheduler_gamma = post_sched_gamma,\
                      init_update_sigma = init_update_sigma, verbose = verbose, per_compute_verbose = per_compute_verbose)

In [ ]:
import torch
a = torch.tensor([1,2,3,4]).float()
idx_1 = a.eq(4)
idx_1.nonzero().item()

In [ ]:
fig, ax = plt.subplots()
ax.plot(loss_array)
ax.set_title("Error over iterations, InstanceId " + str(inst_id))
ax.set_xlabel("Iteration")
ax.set_ylabel("Error")

In [ ]:
# Get sigma with min error
best_sigma = model.update_sigma
test_error = compute_post_pred_error(df_real, df_real_test, scale_df, inst_id, model, likelihood, best_sigma, dual_scale_factor, steep_d_sigmoid_basis, kpi=kpi, use_relu = use_relu, verbose = verbose)
print("Best sigma for training data: ", best_sigma)
print("Test error for best sigma: ", test_error)